In [ ]:
!wget -O dataset.csv ""
!wget -O macro_data_master.csv ""

: 

In [ ]:
working_directory = "/content/"
# %load_ext rpy2.ipython

: 

In [ ]:
import pandas as pd
original_df = pd.read_csv(f"{working_directory}dataset.csv")


: 

In [ ]:
macro_master = pd.read_csv(f"{working_directory}macro_data_master.csv")

: 

In [ ]:
original_df

: 

In [ ]:
original_df.columns = map(str.lower, original_df.columns)
list(original_df.columns),len(list(original_df.columns))

: 

In [ ]:
macro_master

: 

In [ ]:
list(macro_master.columns),len(list(macro_master.columns))

: 

In [ ]:
macro_master.columns = map(str.lower, macro_master.columns)
macro_master.columns = macro_master.columns.str.strip()
macro_master.columns = macro_master.columns.str.replace(' ', '_')
list(macro_master.columns),len(list(macro_master.columns))

: 

In [ ]:
macro_master['date'] = pd.to_datetime(macro_master['date'], format='%m/%d/%Y')
macro_master.set_index('date', inplace=True)

: 

In [ ]:
macro_qtr=macro_master

: 

In [ ]:
macro_qtr

: 

In [ ]:
macro_qtr = macro_qtr.replace(['.'], pd.NA)
macro_qtr

: 

In [ ]:
# convert all string columns to numeric
macro_qtr = macro_qtr.apply(pd.to_numeric, errors='ignore')
macro_qtr

: 

In [ ]:
# fill nan with mean of each quarter based on date
macro_qtr = macro_qtr.groupby(pd.Grouper(freq='Q')).transform(lambda x: x.fillna(x.mean()))

: 

In [ ]:

macro_qtr[macro_qtr.isna().any(axis=1)]


: 

In [ ]:
# convert macro_qtr to quarterly averages based on date
macro_qtr = macro_qtr.groupby(pd.Grouper(freq='Q')).mean()
macro_qtr

: 

In [ ]:
# Add macro_qtr columns to macro_qtr dataframe in the format 2010Q3
macro_qtr["macro_qtr"] = macro_qtr.index.year.astype(str) + "Q" + macro_qtr.index.quarter.astype(str)

: 

In [ ]:
macro_qtr

: 

In [ ]:
# set date back to column
macro_qtr.reset_index(inplace=True)
macro_qtr

: 

In [ ]:
df=original_df
df["datafqtr"]

: 

In [ ]:
# merge macro data with original data on month and year
df = pd.merge(df, macro_qtr, left_on='datafqtr', right_on='macro_qtr', how='left')

: 

In [ ]:
df

: 

In [ ]:
# count all atq values greater less than 1000
df[df['atq'] < 1000].count()

: 

In [ ]:
# Drop all rows with atq< 1000
df = df[df['atq'] < 1000]

: 

In [ ]:

df.shape

: 

In [ ]:
df.isna().sum().sort_values(ascending=False)

: 

In [ ]:
# replace 0 with nan and  drop all nan in revtq
df['revtq'] = df['revtq'].replace(0, pd.NA)
df = df.dropna(subset=['revtq'])

: 

In [ ]:
#drop duplicates
df = df.drop_duplicates()

: 

In [ ]:
df.shape

: 

In [ ]:
# replace 0 with nan and  drop all nan in lctq
df['lctq'] = df['lctq'].replace(0, pd.NA)
df = df.dropna(subset=['lctq'])

: 

In [ ]:
# replace 0 with nan and  drop all nan in ppentq
df['ppentq'] = df['ppentq'].replace(0, pd.NA)
df = df.dropna(subset=['ppentq'])

: 

In [ ]:
df["core"] =  df["cogsq"]/df["revtq"]

: 

In [ ]:
df["opre"]= df["xoprq"]/df["revtq"]


: 

In [ ]:
df["roa"]=df["niq"]/df["atq"]

: 

In [ ]:
df["sata"]=df["revtq"]/df["atq"]

: 

In [ ]:
df["depp"]=df["dpq"]/df["ppentq"]

: 

In [ ]:
df["cacl"]=df["actq"]/df["lctq"]

: 

In [ ]:
#check na values only in core, roa, sata, depp, cacl
df[['core','roa','sata','depp','cacl']].isna().sum()

: 

In [ ]:
df["cacl"]=df["cacl"].fillna(1)

: 

In [ ]:
df["depp"]=df["depp"].fillna(0.025)

: 

In [ ]:
df["core"]=df["core"].fillna(0)
df["opre"]=df["opre"].fillna(0)

: 

In [ ]:
df[['core','roa','sata','depp','cacl','opre']].isna().sum()

: 

In [ ]:
df=df.dropna(how="any", subset=["opepsq"])

: 

In [ ]:
# fill sata and roa with median for na values
df["sata"]=df["sata"].fillna(df["sata"].median())
df["roa"]=df["roa"].fillna(df["roa"].median())

: 

In [ ]:
df.shape

: 

In [ ]:
df["ebitda"]=df["oibdpq"]/df["revtq"]

: 

In [ ]:
#add a lag column for ebitda

df["ebitda_lag"]=df.groupby("gvkey")["ebitda"].shift(1)

: 

In [ ]:
target = df["ebitda_lag"]
feat = df[["depp","core","opre","sata","cacl","roa","opepsq"]]

: 

In [ ]:
from sklearn.model_selection import train_test_split
import xgboost as xgb

X_train, X_test, y_train, y_test = train_test_split(feat, target, test_size=0.3, random_state=42)

: 

In [ ]:
# xgboost regressor
xgb_model = xgb.XGBRegressor(objective="reg:squarederror")
xgb_model.fit(X_train, y_train)

: 

In [ ]:
y_pred = xgb_model.predict(X_test)

: 

In [ ]:

%%R
install.packages("dplyr")
library(dplyr)
install.packages("corrplot") # run if not downloaded before
library(corrplot)

rm(list= ls())
setwd("/content/")
df <- read.csv("final_df.csv") # data after basic cleaning

# change column name
colnames(df)[colnames(df) == "opepsq"] <- "eps"

################## Identify factors we use, deal with NANs #####################
## (1) Identify factors
# Y: outcome last period & outcome
Y <- c("ebitda","ebitda_lag")

# X: name of predictors we use
corp_factors <- c("cogsq", "revtq", "xoprq","eps","niq","atq","dpq","ppentq", "oibdpq", "actq", "lctq")
macro_factors <- c("vix_index", "gold", "cpi", "X3.y_yield", "gdp", "tax_recipts", "oil", "un_em_rate", "taxation", "m2", "energy","b_trade", "pce", "gov_exp", "fed_debt", "n_p_gdp")

X <- append(corp_factors, macro_factors)

# X+Y factors
XY <- append(X,Y)

dataset_X <- df[,X] # datasets of X
dataset_XY <- df[,XY] # datasets of X+Y

## (2) handling with NANs

# see how many NANs existing for each column
apply(dataset_XY, MARGIN = 2, FUN = function(x){sum(is.na(x))})  # "MARGIN" identify row/column we want to apply "FUN"

# drop all NANs, since size of missing values are relatively small
dataset_XY <- dataset_XY[complete.cases(dataset_XY),] # df[a,b]: for df, "a" identifies rows, "b" identifies columns
dataset_X <- dataset_XY[,X]

### (3) data description

# here, draw distribution graph for each variables in dataset_X


# correlation matrix can also be drawn if you like, which shows strength of relationship(in data sense) between variables
par(mfrow=c(1,1), mar=c(4,4,2,0.5))
M = cor(dataset_XY)
corrplot(M, order = "hclust", addrect = 2)
    # from graph, we can see that, (1) our target "ebitda_lag" is only correlated with "ebitda" and itself
                                #  (2) ...

#######################  feature Engneering ###################################
##### 1. make interaction between factors
print(X)
# Create an empty data frame to store the interaction terms
interact_dataset_X <- data.frame(matrix(NA, nrow = nrow(dataset_X), ncol = 0))

# Loop through all pairs of variables and create interaction terms
for (i in 1:(length(X) - 1)) {
  for (j in (i + 1):length(X)) {
    var1 <- X[i]
    var2 <- X[j]
    interaction_name <- paste(var1, var2, sep = "_") # e.g: "gdp_m2"

    # Create the interaction term and store it in the data frame
    interact_dataset_X[interaction_name] <- dataset_X[[var1]] * dataset_X[[var2]]
  }
}

# check if any missing value
apply(interact_dataset_X, 2, function(x){sum(is.na(x))})

##### 2. make square terms of factors
square_dataset_X <- data.frame(matrix(NA, nrow = nrow(dataset_X), ncol = 0))

# Loop through all pairs of variables and create interaction terms
for (i in 1:(length(X))) {
  var <- X[i]
  square_name <- paste0(var,"^2")
  # Create the square term and store it in the data frame
  square_dataset_X[square_name] <- dataset_X[[var]]^2
}

apply(square_dataset_X, 2, function(x){sum(is.na(x))}) # no missing

##### 3. make financial ratios out of factors
ratio_dataset_X <- data.frame(matrix(NA, nrow = nrow(dataset_X), ncol = 0))

ratio_dataset_X$cogs.sale <- dataset_X$cogsq / dataset_X$revtq
ratio_dataset_X$opex.sale <- dataset_X$xoprq / dataset_X$revtq # Operating Expense to Sales Ratio = OPEX / Sales
ratio_dataset_X$roa <- dataset_X$niq / dataset_X$atq # return to assets = Net Income / Assets
ratio_dataset_X$sale.asset <- dataset_X$revtq / dataset_X$atq # Assets Turnover Ratio = Sales / Assets
ratio_dataset_X$ca.cl <- dataset_X$actq / dataset_X$lctq # Current Assets to Current Liability Ratio
ratio_dataset_X$depre.ppe <- dataset_X$dpq / dataset_X$ppentq # Depreciation and Amortisation to PPE ratio

apply(ratio_dataset_X, 2, function(x){sum(is.na(x))}) # no missing

####  Combine datasets
final_df <- data.frame(dataset_XY, ratio_dataset_X, square_dataset_X, interact_dataset_X)


####################### spliting & Benchmark ###################################
### (1) standardization for following model prediction
standar_X <- scale(model.matrix(ebitda_lag ~ . - 1, data = final_df))
standar_X <- subset(standar_X, select = -ebitda) # only standardize predictors

standar_XY <- cbind(standar_X, final_df[,Y])

### (2) Split training & test set
set.seed(114514)
train <- sample(1:nrow(standar_XY), nrow(standar_XY)*4/5) # train: test = 4: 1
test <- (-train)
trainset <- standar_XY[train, ]
testset <- standar_XY[test, ]


x.test <- as.matrix(testset[,!names(standar_XY) %in% Y])
y.test <- as.matrix(testset[, Y[2]])
x.train <- as.matrix(trainset[,!names(standar_XY) %in% Y])
y.train <- as.matrix(trainset[, Y[2]])

benchmark_y <- testset[,Y[1]]

## (3) here, calculate MSE of benchmark_y here.  tips: MSE = E[(benchmark_y - y)^2].
mse_benchmark = mean((benchmark_y-y.test)^2)

: 

: 

: 

: 

: 

: 

: 